# Specs
Based on the TDNN described by Waibel et al. in _Phoneme Recognition using Time Delay Neural Networks_  
On this study, we have a window_size of 3

In [14]:
import sys
sys.path
sys.path.append('../../nn-asr-resources/')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, TimeDistributed, Dense, Reshape, Flatten, Lambda

from time import strftime, gmtime

import os

import data
import train

In [12]:
# Constants
_window_size = 3 # defines the past lookup for determining the following word
_path = "data/simple-examples/data"
_checkpoint_path = "training/conv2d/cp-{epoch:04d}_" + strftime("%H-%M_%d-%m-%Y", gmtime()) + ".ckpt"
_checkpoint_dir = os.path.dirname
data_path = "C:\\Users\\u16164\\Downloads\\dev-clean\\LibriSpeech\\dev-clean"
_mask_value = 2.
features = 13
num_phonemes = 44

In [63]:
def generate_windows(audio, pad_begin, pad_end, window_size, stride):
    padding = [[0, 0], [pad_begin, pad_end], [0, 0]]
    padded_audio = tf.pad(audio, padding)
    timesteps = padded_audio[1]
    windows = tf.map_fn(lambda i: padded_audio[i + window_size],
                        tf.range(timesteps - window_size + 1, delta = stride))
    return windows

In [64]:
# Model creation
model = keras.models.Sequential()
model.add(Lambda(generate_windows,
                 arguments = {"pad_begin":2, "pad_end":2, "window_size":5, "stride":1},
                 name="01_lambda-01"))
model.add(Reshape((-1, 13, 1), input_shape=(None,13), name="02_reshape-01"))
model.add(Conv2D(1,(3,3), name="02_conv2d-01"))
model.add(Conv2D(1,(5,5), name="03_conv2d-02"))
model.add(TimeDistributed(keras.layers.Flatten(), name="04_flatten-01"))
model.add(Dense(num_phonemes + 1, activation=keras.activations.softmax, name="05_dense_output-01"))

In [65]:
print(model.layers[0].input.shape)
# model.summary()

AttributeError: Layer 01_lambda-01 is not connected, no input to return.

In [24]:
trainer = train.Trainer(model, data_path, data_path, 0)
trainer.train(2, optimizer="sgd")

Epoch 1/2


InvalidArgumentError: sequence_length(8) <= 1504
	 [[Node: ctc_4/CTCLoss = CTCLoss[_class=["loc:@training_3/SGD/gradients/ctc_4/CTCLoss_grad/mul"], ctc_merge_repeated=true, ignore_longer_outputs_than_inputs=false, preprocess_collapse_repeated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ctc_4/Log, ctc_4/ToInt64, ctc_4/GatherNd, ctc_4/Squeeze_1)]]